In [ ]:
import time
import numpy as np
import pandas as pd

num_robot = 4
num_table = 9
num_product = 7
map_size = 50

In [ ]:
#状态空间
STATES = []
for a in range(num_product+1):
    for b in range(num_product+1):
        for c in range(num_product+1):
          for d in range(num_product+1):
            for e in range(2):  #31
              for f in range(2):  #32
                for g in range(2):  #41 
                  for h in range(2):  #43
                    for i in range(2):  #52
                      for j in range(2):  #53
                        for k in range(2):  #61
                          for l in range(2):  #62
                            for m in range(2):  #63
                              state = str(a)+str(b)+str(c)+str(d)+str(e)+str(f)+str(g)+str(h)+str(i)+str(j)+str(k)+str(l)+str(m)
                              STATES.append(state)
#可能行动
ACTIONS = []
for i in range(num_table):
  ACTIONS.append('to'+str(i))
#ACTIONS.append('drop')

EPSILON = 0.9
ALPHA = 0.1
GAMMA = 0.9
FRESH_TIME = 0.3
TerminalFlag = "Terminal"

CarryingMatrix = np.zeros((num_robot,1))
StoringMatrix = np.zeros((num_table,num_product))

#工作台位置
table_pos = [
    [1,1],[3,3],[5,5],[7,7],[9,9],[11,11],[13,13],[15,15],[17,17]
]

In [ ]:
def build_q_table(states, actions):
    return pd.DataFrame(
        np.zeros((len(states), len(actions))),
        index = states,
        columns = actions
    )


def choose_action(S, q_table):
    state_table = q_table.loc[S,:]
    if (np.random.uniform() > EPSILON) or ((state_table == 0).all()):
        action_name = np.random.choice(ACTIONS)
    else:
        action_name = state_table.idxmax()
    return action_name


In [ ]:
def get_env_feedback(S, A, i):
    R = 0
    state = list(S)
    for t in range(len(state)):
      state[t] = int(state[t])
    
    if state[0+i] == 0:
      if A == 'to0':
        state[0+i] = 1
        R = -30
      elif A == 'to1': 
        state[0+i] = 2
        R = -44
      elif A == 'to2':
        state[0+i] = 3
        R = -58
    elif state[0+i] == 1:
      if A == 'to8':
        state[0+i] = 0
        R = 60
      elif A == 'to3':
        state[4] = 1
        if state[5] == 1:
          state[5] = 0
          state[4] = 0
          state[0+i] = 4
          R = 60-154
        else:
          R = 60
          state[0+i] = 0
      elif A == 'to4':
        state[6] = 1
        if state[7] == 1:
          state[7] = 0
          state[6] = 0
          state[0+i] = 5
          R = 60-172
        else:
          R = 60
          state[0+i] = 0
    elif state[0+i] == 2:
      if A == 'to8':
        state[0+i] = 0
        R = 76
      elif A == 'to3':
        state[5] = 1
        if state[4] == 1:
          state[5] = 0
          state[4] = 0
          state[0+i] = 4
          R = 76-154
        else:
          R = 76
          state[0+i] = 0
      elif A == 'to5':
        state[8] = 1
        if state[9] == 1:
          state[9] = 0
          state[8] = 0
          state[0+i] = 6
          R = 76-192
        else:
          R = 76
          state[0+i] = 0
    elif state[0+i] == 3:
      if A == 'to8':
        state[0+i] = 0
        R = 92
      elif A == 'to4':
        state[7] = 1
        if state[6] == 1:
          state[6] = 0
          state[7] = 0
          state[0+i] = 4
          R = 92-172
        else:
          R = 92
          state[0+i] = 0
      elif A == 'to5':
        state[9] = 1
        if state[8] == 1:
          state[9] = 0
          state[8] = 0
          state[0+i] = 6
          R = 92-192
        else:
          R = 92
          state[0+i] = 0
    elif state[0+i] == 4:
      if A == 'to8':
        state[0+i] = 0
        R = 225
      elif A == 'to6':
        state[10] = 1
        if state[11] == 1 and state[12] == 1:
          state[10] = state[11] = state[12] = 0
          R = 225-760
          state[0+i] = 7
        else:
          R = 225
          state[0+i] = 0
    elif state[0+i] == 5:
      if A == 'to8':
        state[0+i] = 0
        R = 250
      elif A == 'to6':
        state[11] = 1
        if state[10] == 1 and state[12] == 1:
          state[10] = state[11] = state[12] = 0
          R = 250-760
          state[0+i] = 7
        else:
          R = 250
          state[0+i] = 0
    elif state[0+i] == 6:
      if A == 'to8':
        state[0+i] = 0
        R = 275
      elif A == 'to6':
        state[12] = 1
        if state[10] == 1 and state[11] == 1:
          state[10] = state[11] = state[12] = 0
          R = 275-760
          state[0+i] = 7
        else:
          R = 275
          state[0+i] = 0
    elif state[0+i] == 7:
      if A == 'to7' or A == 'to8':
        state[0+i] = 0
        R = 1050
    
    for i in range(len(state)):
      state[i] = str(state[i])
    S_ = ''.join(state)
    R = -R if R < 0 else R 
    return S_, R

In [ ]:
MAX_EPISODES = 2
def rl():
    Q_table=[]
    for i in range(num_robot):
      table = build_q_table(STATES, ACTIONS)
      Q_table.append(table)
    #Q_table = Q_saved
    for episode in range(MAX_EPISODES):
        step_counter = 0
        S = '0000000000000'
        is_terminated = False
        while not is_terminated:
          for i in range(num_robot):
            A = choose_action(S, Q_table[i])
            S_, R = get_env_feedback(S, A, i)
            q_predict = Q_table[i].loc[S, A]

            if S_ != TerminalFlag:
                q_target = R + GAMMA * Q_table[i].loc[S_,:].max()
            else:
                q_target = R
                is_terminated = True
            
            Q_table[i].loc[S, A] += ALPHA * (q_target - q_predict)
            
            S = S_

          step_counter += 1
          if step_counter % 10000 == 0:
              print('epoch%d,step%d'%(episode,step_counter))
          if step_counter > 99999 :
              break
    return Q_table

Q_table = rl()

epoch0,step10000
epoch0,step20000
epoch0,step30000
epoch0,step40000
epoch0,step50000
epoch0,step60000
epoch0,step70000
epoch0,step80000
epoch0,step90000
epoch0,step100000
epoch1,step10000
epoch1,step20000
epoch1,step30000
epoch1,step40000
epoch1,step50000
epoch1,step60000
epoch1,step70000
epoch1,step80000
epoch1,step90000
epoch1,step100000


In [ ]:
Q_table

[               to0        to1  to2  to3  to4  to5  to6  to7  to8
 0000000000000  3.0  34.392694  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0000000000001  0.0   0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0000000000010  0.0   0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0000000000011  0.0   0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0000000000100  0.0  22.954936  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 ...            ...        ...  ...  ...  ...  ...  ...  ...  ...
 7777111111011  0.0   0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 7777111111100  0.0   0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 7777111111101  0.0   0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 7777111111110  0.0   0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 7777111111111  0.0   0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 
 [2097152 rows x 9 columns],
                to0     to1        to2  to3  to4  to5  to6  to7  to8
 0000000000000  0.0   0.000  37.964344  0.0  0.0  0.0  0.0  0.0  0.0
 0000000000001  0.0   0.000  30.258780 

In [ ]:
def test_action(S,step):
  STEP = []
  R_total = 0
  for i in range(num_robot):
    STEP.append([])
  for i in range(step):
    for i in range(num_robot):
      A = choose_action(S, Q_table[i])
      S_, R = get_env_feedback(S, A, i)
      R_total = R + R_total
      STEP[i].append(A)
      S = S_
  
  for i in range(num_robot):
    print('Robot%d:'%(i+1))
    print(STEP[i])
  return R_total

S0 = '0000000000000'
test_action(S0,10)

Robot1:
['to1', 'to5', 'to1', 'to5', 'to1', 'to4', 'to8', 'to1', 'to8', 'to1']
Robot2:
['to2', 'to5', 'to6', 'to0', 'to1', 'to3', 'to0', 'to4', 'to2', 'to0']
Robot3:
['to2', 'to7', 'to8', 'to0', 'to8', 'to5', 'to2', 'to4', 'to6', 'to8']
Robot4:
['to0', 'to7', 'to3', 'to1', 'to1', 'to3', 'to8', 'to1', 'to8', 'to2']


2413

In [ ]:
x = '123456'
state = list(x)
for i in range(len(state)):
  state[i] = int(state[i])

for i in range(len(state)):
    state[i] = str(state[i])
S_ = ''.join(state)
S_

'123456'